In [1]:
import requests
import json
import pandas as pd

#url = "https://www.weather.gov/source/aprfc/it_2022.json"

#this is the variable that I will store station id, station name, measurement date, and ice thickness
rows=[]
# loop over years range
for year in range(1995, 2024):
    # construct the URL for the current year
    url = f'https://www.weather.gov/source/aprfc/it_{year}.json'

    response = requests.get(url)
    data = json.loads(response.text)
    for items in data:
        #print(items)
        #print("test run!!!!")
        for layer1 in data[items]:
            #print("###################")
            #print("  ",layer1)
            
        # for layer2 in data[items][layer1]:
        #    print("--------------------------------")
        #     print("  ",layer2)
                
            for layer3 in data[items][layer1]["features"]:

                #determine how many measurements were taken at each location
                numOfMeasurements = len(layer3['properties']['data'])


                date_and_measure_list = (layer3['properties']['data'])
                lid = layer3['properties']['lid']
                st_name = layer3['properties']['name']




                """print(lid)
                print(st_name)
                print(layer3['properties']['data'])
                print(numOfMeasurements)
                print(date_and_measure_list)


                print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
                print("     ") """


                #need to create a for loop here that creates multiple rows if there are more than one date/measurement
                n = (numOfMeasurements)
                i=0
                while i < n:
                    rows.append([lid,st_name,year,date_and_measure_list[i][0],date_and_measure_list[i][1]])
                    i+=1
                    
print(rows)                   
               


[['ALSA2', 'Port Alsworth RAWS', 1995, '1995-01-02 00:00:00', '18'], ['BTTA2', 'Koyukuk River nr Bettles', 1995, '1994-12-31 00:00:00', '24'], ['RSHA2', 'Yukon River at Russian Mission', 1995, '1994-12-31 00:00:00', '17'], ['SMLA2', 'Smith Lake Ice', 1995, '1994-12-24 00:00:00', '16'], ['SSLA2', 'Snowshoe Lake Ice near Nelchina', 1995, '1995-01-02 00:00:00', '36'], ['TALA2', 'Tanana', 1995, '1995-01-04 00:00:00', '20'], ['YEAA2', 'Yukon River at Eagle', 1995, '1994-12-31 00:00:00', '27'], ['ALSA2', 'Port Alsworth RAWS', 1995, '1995-01-31 00:00:00', '26'], ['BTTA2', 'Koyukuk River nr Bettles', 1995, '1995-01-31 00:00:00', '30'], ['CTLA2', 'Cottonwood Lake', 1995, '1995-02-02 00:00:00', '34'], ['LVRA2', 'Big River Lakes Ice', 1995, '1995-02-03 00:00:00', '38'], ['PAEC', 'Chulitna River Lodge', 1995, '1995-01-30 00:00:00', '36'], ['RSHA2', 'Yukon River at Russian Mission', 1995, '1995-01-31 00:00:00', '26'], ['SMLA2', 'Smith Lake Ice', 1995, '1995-02-02 00:00:00', '20'], ['SSLA2', 'Snowsh

In [2]:
df=pd.DataFrame(rows, columns=["Lid","Name","Year","Date","Measurement"])
pd.set_option('display.width',1000)
print(df)

        Lid                             Name  Year                 Date Measurement
0     ALSA2               Port Alsworth RAWS  1995  1995-01-02 00:00:00          18
1     BTTA2         Koyukuk River nr Bettles  1995  1994-12-31 00:00:00          24
2     RSHA2   Yukon River at Russian Mission  1995  1994-12-31 00:00:00          17
3     SMLA2                   Smith Lake Ice  1995  1994-12-24 00:00:00          16
4     SSLA2  Snowshoe Lake Ice near Nelchina  1995  1995-01-02 00:00:00          36
...     ...                              ...   ...                  ...         ...
4741  TFIA2    Tanana River at Fairbanks Ice  2023  2023-04-04 16:01:00          34
4742  MXYA2    Kennicott River near McCarthy  2023  2023-04-17 14:00:00          25
4743  MXYA2    Kennicott River near McCarthy  2023  2023-04-18 14:00:00          24
4744  MXYA2    Kennicott River near McCarthy  2023  2023-04-19 14:00:00          24
4745  MXYA2    Kennicott River near McCarthy  2023  2023-04-20 14:00:00     

In [3]:
#df_expanded = df.assign(data_dict=df['data_dict'].apply(pd.Series)).explode('data_dict').reset_index(drop=True)